In [1]:
#!pip install torchvision

# DATASE and DATALOADER

더 나은 가독성과 모듈성을 위해 데이터셋 코드와 모델학습 코드를 분리하는것이 이상적.
It is ideal to separate dateset codes and model training codes for the sake of better readability and modualarity

Pytorch는 torch.util.data.DataLoader와 torch.utils.data.Dataset 를 제공하여 pre-loaded 데이터셋과 가지고 있는 데이터를 사용 가능 



## 데이터셋 불러오기

Fashion-MNIST 데이터셋
- 60,000 training images
- 10,000 test images

각각의 이미지는 grayscale 28x28 이미지와 10의 label로 구성 

root : 학습/테스트 데이터가 저장되는 경로 
train : 학습용 or 테스트용 데이터셋 여부 지정 
download=True : root에 데이터가 없는 경우 인터넷에서 다운로드 
transform과 target_transform은 feature와 label 변형을 지정

In [2]:
import torch 
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt 

In [3]:
training_data = datasets.FashionMNIST(
root='data',
train=True,
download=True,
transform=ToTensor())

In [4]:
test_data = datasets.FashionMNIST(
root='data',
train=False,
download=True,
transform=ToTensor()
)

## 데이터셋 순회후 시각화


In [6]:
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt" ,
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

In [ ]:
figure = plt.figure(figsize=(8,8))
cols, rows = 3, 3
for i in range(1, cols*rows+1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img.squeeze(),cmap='gray')
plt.show()

## 파일에서 사용자 정의 데이터 셋 만들기 

In [ ]:
import os 
import pandas as pd 
from torchvision.io import read_image

class CustomImageDataset(Dataset):
    # 함수는 dataset객체가 생성될 떄 한번 만 실행. 여기서는 annotation_file이 포함된 디렉토리와 transform을 초기화
    def __init__(self. annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annptations_file, anmes=['file_name', 'label'])
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        
    # 데이터셋의 셈플 개수 반환
    def __len__(self):
        return len(self.img_labels)
    
    # 주어진 인덱스 idx에 해당하는 샘픙르 데이터셋에서 불러오고 반환.
    def __getitem__(self,idx):
        # 1. 인덱스 기반으로 디스크에서 이미지 위치 식별
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        # 2. read_image를 사용하여 이미지를 tensor롤 변환
        image = read_image(img_path)
        # 3. csv데이터로부터 해당하는 라벨을가져오고
        label = self.img_labels.iloc[idx, 1]
        # 4. 해당한다면, transform함수를 호출하여 텐서 이미지와 라벨을 python dict령으로 반환
        if self.transfoem:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
            
        return image, label

## DataLoader로 학습용 데이터 준비하기 

Dataset은 feature를 가져오고 각각의 샘플에 label값을 지정하는 작업을 동시에 수행. 
모델 학습시, 통상 minibatch로 전달하고 epoch마다 데이터를 섞어 overfitting을 방지하며, python의 multiporcessing을 사용하여 데이터 검색 속도를 높임 

DataLoader는 간단한 API로 복잡한 과정들을 추상화한 iterable 객체

In [5]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [ ]:
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape:{train_features.size()}")
print(f"Labels batch shape:{train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap='gray')
plt.show()
print(f"Label: {label}")

Feature batch shape:torch.Size([64, 1, 28, 28])
Labels batch shape:torch.Size([64])
